# Pizza Place Sales Analysis

In [ ]:
### Importing the needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Loading the data into a pandas DataFrame
data_dictionary = pd.read_csv('Downloads/data_dictionary.csv', encoding='latin-1')
order_details= pd.read_csv('Downloads/order_details.csv', encoding='latin-1')
orders= pd.read_csv('Downloads/orders.csv', encoding='latin-1')
pizza_types= pd.read_csv('Downloads/pizza_types.csv', encoding='latin-1')
pizzas= pd.read_csv('Downloads/pizzas.csv', encoding='latin-1')

In [ ]:
### Previewing the Datasets
orders.head()

In [ ]:
order_details.head()

In [ ]:
pizzas.head()

In [ ]:
pizza_types.head()

In [ ]:
data_dictionary.head()

In [ ]:
#Data cleaning
# Check info
orders.info()

# Check missing values
orders.isnull().sum()

In [ ]:
# Check info
pizza_types.info()

# Check missing values
pizza_types.isnull().sum()

In [ ]:
# Check info
pizzas.info()

# Check missing values
pizzas.isnull().sum()

In [ ]:
# Check info
order_details.info()

# Check missing values
order_details.isnull().sum()

In [ ]:
### Merging the Datasets
# Merging order_details with pizzas
order_details = order_details.merge(pizzas, on='pizza_id', how='left')
# Merging order_details with pizza_types
order_details = order_details.merge(pizza_types, on='pizza_type_id', how='left')
# Merging order_details with orders
full_data = order_details.merge(orders, on='order_id', how='left')

In [ ]:
# Converting the date and time columns
full_data['date'] = pd.to_datetime(full_data['date'])
full_data['time'] = pd.to_datetime(full_data['time'], format='%H:%M:%S').dt.time

full_data.head()

## Exploratory Data Analysis (EDA)

In [ ]:
#1. Total Revenue/Sales
total_revenue = (full_data['price'] * full_data['quantity']).sum()
print(f"The total Revenue was ${total_revenue:,.2f}")

#2. Total Quantity Sold
total_quantity = full_data['quantity'].sum()
print(f"The total quantity sold was {total_quantity}")

#3. Total Orders
total_orders = orders['order_id'].nunique()
print(f"The total Orders was {total_orders}")

#4. Number of Pizza Types Sold
pizza_types_sold = full_data['pizza_type_id'].nunique()
print(f"The number of pizza types sold was {pizza_types_sold}")

#5. Average Price of Pizzas
average_price = pizzas['price'].mean()
print(f"The average pizza price was ${average_price:.2f}")

#6. Peak Hours of Sales
full_data['hour'] = pd.to_datetime(full_data['time'].astype(str)).dt.hour

plt.figure(figsize=(10,6))
sns.countplot(x='hour', data=full_data, palette='viridis')
plt.title('Peak Hours of Sales')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Orders')
plt.show()

#7. Sales by Day of the Week
full_data['day_of_week'] = full_data['date'].dt.day_name()

sales_by_day = full_data.groupby('day_of_week').apply(lambda x: (x['price'] * x['quantity']).sum())
sales_by_day = sales_by_day.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

sales_by_day.plot(kind='bar', figsize=(10,6), color='skyblue')
plt.title('Total Sales by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Total Sales ($)')
plt.show()

#8. Top 5 Bestselling Pizzas
bestselling_pizzas = full_data.groupby('pizza_type_id')['quantity'].sum().sort_values(ascending=False).head(5)

bestselling_pizzas.plot(kind='bar', figsize=(10,6), color='salmon')
plt.title('Top 5 Bestselling Pizzas')
plt.xlabel('Pizza Type')
plt.ylabel('Quantity Sold')
plt.show()

#9. Monthly Sales and Trend Analysis
full_data['month'] = full_data['date'].dt.month_name()
monthly_sales = full_data.groupby('month').apply(lambda x: (x['price'] * x['quantity']).sum())

# Order months
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_sales = monthly_sales.reindex(month_order)

monthly_sales.plot(kind='line', marker='o', figsize=(12,6))
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Sales ($)')
plt.grid(True)
plt.show()

#10. Underperforming Pizzas
pizza_sales = full_data.groupby('pizza_type_id')['quantity'].sum().sort_values()

# Bottom 5 Pizzas
bottom_pizzas = pizza_sales.head(5)

bottom_pizzas.plot(kind='barh', figsize=(10,6), color='lightcoral')
plt.title('Bottom 5 Underperforming Pizzas')
plt.xlabel('Quantity Sold')
plt.ylabel('Pizza Type')
plt.show()

# Additional Insights

#11. Pizza size popularity 
size_sales = full_data.groupby('size')['quantity'].sum().sort_values()

# Plot
size_sales.plot(kind='barh', color='skyblue')
plt.title('Sales by Pizza Size')
plt.xlabel('Total Pizzas Sold')
plt.ylabel('Pizza Size')
plt.show()

#12. Popular ingredients 
# Split ingredients into separate words
ingredient_list = full_data['ingredients'].str.split(',').explode()

top_ingredients = ingredient_list.value_counts().head(10)

# Plot
sns.barplot(y=top_ingredients.index, x=top_ingredients.values, palette='coolwarm')
plt.title('Top 10 Most Common Ingredients')
plt.xlabel('Count')
plt.ylabel('Ingredient')
plt.show()

#13. Weekly Sales Trend (Seasonality)
# Extract week day
full_data['weekday'] = full_data['date'].dt.day_name()

weekday_sales = full_data.groupby('weekday')['quantity'].sum().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
])

# Plot
sns.lineplot(x=weekday_sales.index, y=weekday_sales.values, marker='o')
plt.title('Sales Trend by Day of the Week')
plt.ylabel('Total Pizzas Sold')
plt.xlabel('Weekday')
plt.xticks(rotation=45)
plt.show()


#14. Top 10 best selling pizzas
top_pizzas = full_data.groupby('name')['quantity'].sum().sort_values(ascending=False).head(10)

# Plot
sns.barplot(x=top_pizzas.values, y=top_pizzas.index, palette="viridis")
plt.title('Top 10 Best Selling Pizzas')
plt.xlabel('Total Quantity Sold')
plt.ylabel('Pizza Name')
plt.show()
